In [4]:
!pip install numpy
!pip install gym
import numpy as np
import gym
import random
import time
from IPython.display import clear_output
from gym.envs.toy_text.frozen_lake import generate_random_map


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\clayb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\clayb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
env = gym.make('FrozenLake-v1', render_mode='ansi')

In [11]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [13]:
def load_frozen_lake(desc=None, map_name=None, is_slippery=False):
    """
    Load the FrozenLake environment from OpenAI's gym.

    Parameters:
    desc -- None or a list of lists containing a custom
    description of the map to load for the environment
    map_name -- None or a string containing the pre-made map to load
    is_slippery -- a boolean to determine if the ice is slippery

    If both desc and map_name are None, the environment will
    load a randomly generated 8x8 map.

    Returns:
    The FrozenLake environment.
    """
    if desc is None and map_name is None:
        desc = generate_random_map(size=8)

    return gym.make('FrozenLake-v0', desc=desc,
                    map_name=map_name, is_slippery=is_slippery)

In [14]:
def q_init(env):
    """Initialize Q-table to zeros."""
    return np.zeros([env.observation_space.n, env.action_space.n])

In [10]:
def epsilon_greedy(Q, state, epsilon):
    """Epsilon-greedy policy."""
    if random.uniform(0, 1) < epsilon:
        action = env.action_space.sample()  # Explore action space
    else:
        action = np.argmax(Q[state])  # Exploit learned values
    return action

In [15]:
def train(env, Q, episodes=5000, max_steps=100,
          alpha=0.1, gamma=0.99, epsilon=1,
          min_epsilon=0.1, epsilon_decay=0.05):
    """Train agent to learn Q-values."""
    total_rewards = []
    for episode in range(episodes):
        state = env.reset()
        rewards = 0
        for step in range(max_steps):
            action = epsilon_greedy(Q, state, epsilon)
            new_state, reward, done, info = env.step(action)
            if done and reward == 0:
                reward = -1
            Q[state, action] = (1 - alpha) * Q[state, action] + alpha * (reward + gamma * np.max(Q[new_state]))
            state = new_state
            rewards += reward
            if done:
                break
        epsilon = min_epsilon + (epsilon - min_epsilon) * np.exp(-epsilon_decay * episode)
        total_rewards.append(rewards)
    return Q, total_rewards

In [16]:
def play(env, Q, max_steps=100):
    """Play agent with learned Q-values."""
    state = env.reset()
    total_rewards = 0
    for step in range(max_steps):
        action = np.argmax(Q[state])
        new_state, reward, done, info = env.step(action)
        env.render()
        state = new_state
        total_rewards += reward
        if done:
            break
    return total_rewards